# Crawling Pipeline

#### 1. TRADE_COUNTRYCD
#### 2. TRADE_HSCD
#### 3. TRADE_HSCD_COUNTRYCD
#### 4. HSCD_NAME

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings; warnings.filterwarnings('ignore')

# 크롤링
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import re
import time
import openpyxl


train = pd.read_csv('kotra_train.csv')
train

<font color='green'>

## 1. TRADE_COUNTRYCD

In [ ]:
# ----- TRADE_COUNTRYCD_infos -----
years = [2013, 2015, 2016]
Reporters = train.COUNTRYNM.unique().tolist()
partners = 'World'
Trade_flows = 'Import'
commodity_codes = 'TOTAL'

# ----- Crawling Start -----

dfs = []
for year in years:
    print(f'##### {year} years #####')
    trade_countrycd_lst = []
    
    year_reset = dr.find_element_by_css_selector(f'div#s2id_periods li.select2-search-choice a')
    year_reset.click()
    time.sleep(0.5)

    year_input = dr.find_element_by_css_selector('div#s2id_periods li.select2-search-field  input')
    year_input.send_keys(f'{year}')
    year_input.send_keys(Keys.RETURN)
    
    # partners
    partners_reset = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-choice a')
    partners_reset.click()
    time.sleep(0.5)

    partners_input = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-field  input')
    partners_input.send_keys(f'{partners}')
    partners_input.send_keys(Keys.RETURN)

    # Trade flows
    flows_reset = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-choice a')
    flows_reset.click()
    time.sleep(1)

    flows_input = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-field  input')
    flows_input.send_keys(f'{Trade_flows}')
    flows_input.send_keys(Keys.RETURN)   

    # HS (as reported) commodity codes
    HSCD_codes_reset = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-choice a')
    HSCD_codes_reset.click()
    time.sleep(0.5)

    HSCD_codes_input = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-field  input')
    HSCD_codes_input.send_keys(f'{commodity_codes}')
    HSCD_codes_input.send_keys(Keys.RETURN)
    HSCD_codes_input.send_keys(Keys.ESCAPE)    
    
    for reporter in Reporters:
        print(reporter, end=', ')
        reporters_reset = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-choice a')
        reporters_reset.click()
        time.sleep(0.5)

        reporters_input = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-field  input')
        reporters_input.send_keys(f'{reporter}')
        reporters_input.send_keys(Keys.RETURN)
        
        # see the results
        time.sleep(0.5)
        see_the_results = dr.find_element_by_css_selector('div.col-md-12 button#preview')
        see_the_results.click()
        time.sleep(4)
        
        # get TRADE_COUNTRYCD
        try:
            trade_countrycd = dr.find_element_by_css_selector('tr.odd td.dt-right').text.lstrip('$').replace(',','')
            trade_countrycd_lst.append(trade_countrycd)
            print(trade_countrycd)
            time.sleep(1)
        except:
            trade_countrycd_lst.append(np.nan)
            print(np.nan)
            time.sleep(1)
            
    df = pd.DataFrame(index=train.COUNTRYNM.unique().tolist(), data=trade_countrycd_lst, columns=[f'TRADE_COUNTRYCD_{year}'])
    dfs.append(df)
    
dfs

print(len(years))
print(len(dfs))

trade_countrycd_data = pd.concat(dfs, axis=1); display(trade_countrycd_data)
trade_countrycd_data.to_csv('trade_countrycd_data.csv', index=False)

<font color='green'>

## 2. TRADE_HSCD

TRADE_HSCD_2015

In [60]:
sheets_sum = []
browser = Chrome()
browser.get('https://comtrade.un.org/data')
time.sleep(20)

browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-field input').send_keys('2015')
browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-field input').send_keys(Keys.RETURN)
browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-choice a').click()

browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-field input').send_keys('Import')
browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-field input').send_keys(Keys.RETURN)
browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-choice a').click()

for i in HSCD:
    browser.find_element_by_css_selector('#s2id_autogen5').send_keys(f'{i}') 
    browser.find_element_by_css_selector('#s2id_autogen5').send_keys(Keys.RETURN)
    browser.find_element_by_css_selector('#s2id_itemCodes ul li.select2-search-choice a').click()

    browser.find_element_by_css_selector('button#preview.btn.btn-success').click()
    time.sleep(60)
    browser.find_element_by_css_selector('button#preview.btn.btn-success').click()
    time.sleep(60)

    browser.find_element_by_css_selector('#preview-table_length label select option:nth-child(4)').click()
    time.sleep(1)

    iteration = browser.find_element_by_css_selector('#displayed-records').text
    iteration = int(iteration.split(' ')[0])

    sheets = []
    for j in range(iteration):
        trade_hscd = browser.find_element_by_css_selector(f'#preview-table tbody tr:nth-child({j+1}) td:nth-child(6)').text
        sheets.append(trade_hscd)

    sheet_sum = 0
    for k in range(len(sheets)):
        sheet_sum += (int(re.sub('[$,]', '', sheets[k])))
    sheets_sum.append(sheet_sum)

In [93]:
TH_2015 = pd.DataFrame(sheets_sum).rename(columns={0:'TRADE_HSCD_2015'})

TRADE_HSCD_2016

In [27]:
sheets_sum = []
browser = Chrome()
browser.get('https://comtrade.un.org/data')
time.sleep(7)

browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-field input').send_keys('2016')
browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-field input').send_keys(Keys.RETURN)
browser.find_element_by_css_selector('#s2id_periods ul li.select2-search-choice a').click()

browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-field input').send_keys('Import')
browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-field input').send_keys(Keys.RETURN)
browser.find_element_by_css_selector('#s2id_tradeFlow ul li.select2-search-choice a').click()

for i in HSCD:
    browser.find_element_by_css_selector('#s2id_autogen5').send_keys(f'{i}') 
    browser.find_element_by_css_selector('#s2id_autogen5').send_keys(Keys.RETURN)
    browser.find_element_by_css_selector('#s2id_itemCodes ul li.select2-search-choice a').click()

    browser.find_element_by_css_selector('button#preview.btn.btn-success').click()
    time.sleep(60)
    browser.find_element_by_css_selector('button#preview.btn.btn-success').click()
    time.sleep(60)

    browser.find_element_by_css_selector('#preview-table_length label select option:nth-child(4)').click()
    time.sleep(1)

    iteration = browser.find_element_by_css_selector('#displayed-records').text
    iteration = int(iteration.split(' ')[0])

    sheets = []
    for j in range(iteration):
        trade_hscd = browser.find_element_by_css_selector(f'#preview-table tbody tr:nth-child({j+1}) td:nth-child(6)').text
        sheets.append(trade_hscd)

    sheet_sum = 0
    for k in range(len(sheets)):
        sheet_sum += (int(re.sub('[$,]', '', sheets[k])))
    sheets_sum.append(sheet_sum)

In [32]:
TH_2016 = pd.DataFrame(sheets_sum).rename(columns={0:'TRADE_HSCD_2016'})

In [39]:
pd.concat([TH_2016, TH_2015], axis=1).to_csv('TRADE_HSCD_2015~2016.csv', index=False)

<font color='green'>

## 3 .TRADE_HSCD_COUNTRYCD

In [ ]:
dr = Chrome()
dr.get('https://comtrade.un.org/data')
time.sleep(1.5)# ----- TRADE_HSCD_COUNTRYCD_infos -----
years = [i for i in range(2012, 2017)]
Reporters = 'All'
partners = 'World'
Trade_flows = 'Import'
commodity_codes = train.HSCD.unique().tolist()
#commo_num = len(commodity_codes)

In [ ]:
# ----- Crawling Start -----

train_hscd_countrycd_dfs_all = []
for year in years:
    print(f'##### {year} years #####')
    train_hscd_countrycd_dfs = []
    num = 0

    year_reset = dr.find_element_by_css_selector(f'div#s2id_periods li.select2-search-choice a')
    year_reset.click()
    time.sleep(0.5)

    year_input = dr.find_element_by_css_selector('div#s2id_periods li.select2-search-field  input')
    year_input.send_keys(f'{year}')
    year_input.send_keys(Keys.RETURN)
    
    # partners
    partners_reset = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-choice a')
    partners_reset.click()
    time.sleep(0.5)

    partners_input = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-field  input')
    partners_input.send_keys(f'{partners}')
    partners_input.send_keys(Keys.RETURN)

    # Trade flows
    flows_reset = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-choice a')
    flows_reset.click()
    time.sleep(1)

    flows_input = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-field  input')
    flows_input.send_keys(f'{Trade_flows}')
    flows_input.send_keys(Keys.RETURN)   

    # Reporters
    reporters_reset = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-choice a')
    reporters_reset.click()
    time.sleep(0.5)

    reporters_input = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-field  input')
    reporters_input.send_keys(f'{Reporters}')
    reporters_input.send_keys(Keys.RETURN)
    
    for commodity_code in commodity_codes:
        reporter_lst = []
        commodity_lst = []
        trade_hscd_countrycd_lst = []
    
        if num != 0 and num%10 == 0:
            trade_hscd_countrycd_dfs_concat = pd.concat(train_hscd_countrycd_dfs, axis=0);
            print(f'trade_hscd_countrycd_dfs_concat until {num}')
            display(trade_hscd_countrycd_dfs_concat)
            train_hscd_countrycd_dfs_all.append(trade_hscd_countrycd_dfs_concat)
            train_hscd_countrycd_dfs = []
            time.sleep(1.5)
            #time.sleep(60 * 2)
            
        # HS (as reported) commodity codes
        HSCD_codes_reset = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-choice a')
        HSCD_codes_reset.click()
        time.sleep(0.5)

        HSCD_codes_input = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-field  input')
        HSCD_codes_input.send_keys(f'{commodity_code}')
        HSCD_codes_input.send_keys(Keys.RETURN)
        HSCD_codes_input.send_keys(Keys.ESCAPE)    

        # see the results
        time.sleep(0.5)
        see_the_results = dr.find_element_by_css_selector('div.col-md-12 button#preview')
        time.sleep(0.5)
        see_the_results.click()
        time.sleep(0.5)
        try:
            see_the_results.click()
        except:
            pass
        time.sleep(18)
        
        # view 500
        try: # if need to wait for few time due to server
            time.sleep(1)
            show_entries = dr.find_element_by_css_selector('div#preview-table_length select')
            show_entries.click()
        except:
            time.sleep(60 * 10) # wait 6 minutes and access to results again
            see_the_results = dr.find_element_by_css_selector('div.col-md-12 button#preview')
            time.sleep(0.5)
            see_the_results.click()
            time.sleep(0.5)
            show_entries = dr.find_element_by_css_selector('div#preview-table_length select')
            show_entries.click()
        for i in range(3):
            show_entries.send_keys(Keys.ARROW_DOWN)
        show_entries.send_keys(Keys.RETURN)
        time.sleep(1)
            
        # get trade_hscd_country_cd
        infos = dr.find_elements_by_css_selector('tbody tr')
        for info in infos:
            try:
                country_name = info.find_element_by_css_selector('td:nth-of-type(3)').text
                reporter_lst.append(country_name)
            except:
                reporter_lst.append(np.nan)
            try:
                commodity = info.find_element_by_css_selector('td:nth-of-type(5)').text
                commodity_lst.append(commodity)
            except:
                commodity_lst.append(commodity_code)
            try:
                trade_hscd_country_cd = info.find_element_by_css_selector('td:nth-of-type(6)').text.lstrip('$').replace(',','')
                trade_hscd_countrycd_lst.append(trade_hscd_country_cd)
            except:
                trade_hscd_countrycd_lst.append(np.nan)
            
        time.sleep(1.5)    
        df = pd.DataFrame({'COUNTRYNM':reporter_lst, 'HSCD': commodity_lst, f'TRADE_HSCD_COUNTRYCD_{year}': trade_hscd_countrycd_lst})
        num += 1
        print(num); display(df)
        train_hscd_countrycd_dfs.append(df)
        time.sleep(1)
    
    
train_hscd_countrycd_dfs_all_data = pd.concat(train_hscd_countrycd_dfs_all, axis=0);
display(train_hscd_countrycd_dfs_all_data)
train_hscd_countrycd_dfs_all_data.to_csv('trade_hscd_countrycd_2014.csv', index=False)

<font color='green'>

## 4 .HSCD_name

In [ ]:
dr = Chrome()
dr.get('https://comtrade.un.org/data')
time.sleep(1.5)

# ----- TRADE_COUNTRYCD_infos -----
years = '2015'
Reporters = 'All'
partners = 'World'
Trade_flows = 'Import'
commodity_codes = train.HSCD.unique().tolist()
#commodity_codes_from420 = commodity_codes[420:]

# year
year_reset = dr.find_element_by_css_selector(f'div#s2id_periods li.select2-search-choice a')
year_reset.click()
time.sleep(0.5)
year_input = dr.find_element_by_css_selector('div#s2id_periods li.select2-search-field  input')
year_input.send_keys(f'{years}')
year_input.send_keys(Keys.RETURN)

# Reporters
reporters_reset = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-choice a')
reporters_reset.click()
time.sleep(0.5)
reporters_input = dr.find_element_by_css_selector('div#s2id_reporters li.select2-search-field  input')
reporters_input.send_keys(f'{Reporters}')
reporters_input.send_keys(Keys.RETURN)

# partners
partners_reset = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-choice a')
partners_reset.click()
time.sleep(0.5)
partners_input = dr.find_element_by_css_selector('div#s2id_partners li.select2-search-field  input')
partners_input.send_keys(f'{partners}')
partners_input.send_keys(Keys.RETURN)

# Trade flows
flows_reset = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-choice a')
flows_reset.click()
time.sleep(1)
flows_input = dr.find_element_by_css_selector('div#s2id_tradeFlow li.select2-search-field  input')
flows_input.send_keys(f'{Trade_flows}')
flows_input.send_keys(Keys.RETURN)   

In [ ]:
# HS (as reported) commodity codes
trade_value_sum_lst = []
count = 0
for commodity in commodity_codes:        
    HSCD_codes_reset = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-choice a')
    HSCD_codes_reset.click()
    time.sleep(0.5)
    HSCD_codes_input = dr.find_element_by_css_selector('div#s2id_itemCodes li.select2-search-field  input')
    HSCD_codes_input.send_keys(f'{commodity}')
    HSCD_codes_input.send_keys(Keys.RETURN)
    HSCD_codes_input.send_keys(Keys.ESCAPE) 
    time.sleep(0.3)

    # see the results
    get_data = dr.find_element_by_css_selector('div.col-md-12 button#preview')
    get_data.click()
    time.sleep(50)
    
    # view 500
    try: # if need to wait for few time due to server
        time.sleep(1)
        show_entries = dr.find_element_by_css_selector('div#preview-table_length select')
        show_entries.click()
    except:
        time.sleep(60 * 10) # wait 6 minutes and access to results again
        see_the_results = dr.find_element_by_css_selector('div.col-md-12 button#preview')
        print('10Ms wait...')
        time.sleep(0.5)
        see_the_results.click()
        time.sleep(0.5)
        show_entries = dr.find_element_by_css_selector('div#preview-table_length select')
        show_entries.click()
    for i in range(3):
        show_entries.send_keys(Keys.ARROW_DOWN)
    show_entries.send_keys(Keys.RETURN)
    time.sleep(1)    
    
    
    # get trade_hscd_country_cd
    trade_value_lst = []
    infos = dr.find_elements_by_css_selector('tbody tr')
    for info in infos:
        try:
            trade_value = info.find_element_by_css_selector('td:nth-of-type(6)').text.lstrip('$').replace(',','')
            trade_value_lst.append(trade_value)
        except:
            trade_value_lst.append(np.nan)
    try:        
        trade_value_sum = sum([int(num) for num in trade_value_lst])
        trade_value_sum_lst.append(trade_value_sum)
    except:
        trade_value_sum_lst.append(np.nan)
        
    count += 1
    if count != 400 and count % 5 == 0:
        print(count)
    time.sleep(1)
    
    
    